# DAY 2 : RAG

## Passing documents & citations

### Supported models (https://docs.oracle.com/en-us/iaas/Content/generative-ai/chat-models.htm) 
- cohere.command-r-08-2024
- cohere.command-r-16k
- cohere.command-r-plus
- cohere.command-r-plus-08-2024
- meta.llama-3.1-405b-instruct
- meta.llama-3.1-70b-instruct
- meta.llama-3.2-90b-vision-instruct
  
Questions use #generative-ai-users  or ##igiu-innovation-lab slack channels 

### Set up variables

In [12]:
# set up the  variables


from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import *
import oci

#####
#Setup
#Change the compartmentid to yhe ocid of your compartment
#Change the profile if needed
#####

CONFIG_PROFILE = "AISANDBOX"
compartmentId= "ocid1.compartment.oc1..aaaaaaaaxj6fuodcmai6n6z5yyqif6a36ewfmmovn42red37ml3wxlehjmga" 
llm_service_endpoint= "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

LLM_MODEL = "cohere.command-r-plus-08-2024" 
PREAMBLE = """
        you always answer in a one stanza poem.
"""
MESSAGE = """
        "What companies offer database services"
"""

### load the profile

In [13]:
# oci key enabled for api access
config = oci.config.from_file('~/.oci/cionfig', CONFIG_PROFILE)

In [14]:
### set up Chat Request

In [15]:


# chat request      
llm_chat_request = CohereChatRequest()
#llm_chat_request.preamble_override = PREAMBLE 
llm_chat_request.message = MESSAGE
llm_chat_request.is_stream = False 
llm_chat_request.max_tokens = 500 # max token to generate, can lead to incomplete responses
llm_chat_request.temperature = 1.0 # higer value menas more randon, defaul = 0.3
#llm_chat_request.seed = 7555 # makes the best effort to make answer determininstic , not gaureented 
llm_chat_request.top_p = 0.7  # ensures only tokens with toptal probabely of p are considered, max value = 0.99, min 0.01, default 0.75
llm_chat_request.top_k = 0  #Ensures that only top k tokens are considered, 0 turns it off, max = 500
llm_chat_request.frequency_penalty = 0.0 # reduces the repeatedness of tokens max value 1.9=0, min 0,0



### Add documents

In [16]:
docs = [
    {
        "title": "Oracle",
        "snippet": "Oracle database services and products offer customers cost-optimized and high-performance versions of Oracle Database, the world's leading converged, multi-model database management system, as well as in-memory, NoSQL and MySQL databases. Oracle Autonomous Database, available on premises via Oracle Cloud@Customer or in the Oracle Cloud Infrastructure, enables customers to simplify relational database environments and reduce management workloads.",
        "website": "https://www.oracle.com/database"
    },
      {
        "title": "Amazon",
        "snippet": """ AWS provides the broadest selection of purpose-built databases allowing you to save, grow, and innovate faster.
Purpose Built
Choose from 15+ purpose-built database engines including relational, key-value, document, in-memory, graph, time series, wide column, and ledger databases.
Performance at Scale
Get relational databases that are 3-5X faster than popular alternatives, or non-relational databases that give you microsecond to sub-millisecond latency.
Fully Managed
AWS continuously monitors your clusters to keep your workloads running with self-healing storage and automated scaling, so that you can focus on application development.
Secure & Highly Available
AWS databases are built for business-critical, enterprise workloads, offering high availability, reliability, and security.
""",
        "website": "https://aws.amazon.com/free/database/e"
    }
]

#llm_chat_request.documents = docs

## Set up chat details

In [17]:


# set up chat details
chat_detail = ChatDetails()
chat_detail.serving_mode = OnDemandServingMode(model_id=LLM_MODEL)
chat_detail.compartment_id = compartmentId
chat_detail.chat_request = llm_chat_request


### get the LLM client 

In [18]:

# set up the LLM client 
llm_client = GenerativeAiInferenceClient(
                config=config,
                service_endpoint=llm_service_endpoint,
                retry_strategy=oci.retry.NoneRetryStrategy(),
                timeout=(10,240))

### Ask the question 

In [19]:
#llm_chat_request.seed = 7555 # trting changing to see if we can reproduce teh opriginal response
llm_response = llm_client.chat(chat_detail)
llm_text = llm_response.data.chat_response.text
        
print (llm_text)

There are numerous companies that offer database services, providing various solutions for data storage, management, and analytics. Here are some prominent ones:

- **Amazon Web Services (AWS)**: AWS is a leading cloud computing platform that offers a wide range of services, including database services. It provides various options such as Amazon RDS (Relational Database Service), Amazon DynamoDB (a NoSQL database), Amazon Redshift (for data warehousing), and more. AWS is known for its scalability, flexibility, and integration with other AWS services.

- **Microsoft Azure**: Microsoft's cloud computing platform offers a comprehensive suite of database services. This includes Azure SQL Database (a managed relational database), Azure Cosmos DB (a globally distributed NoSQL database), Azure Database for MySQL and PostgreSQL, and Azure Synapse Analytics for data warehousing and big data processing.

- **Google Cloud Platform (GCP)**: GCP provides database solutions as part of its cloud serv

### update the history 

In [20]:
# update history 
previous_chat_message = oci.generative_ai_inference.models.CohereUserMessage(message="Tell me something about Oracle.")
previous_chat_reply = oci.generative_ai_inference.models.CohereChatBotMessage(message="Oracle is one of the largest vendors in the enterprise IT market and the shorthand name of its flagship product. The database software sits at the center of many corporate IT")
        
llm_chat_request.chat_history =  [previous_chat_message, previous_chat_reply]

### ask the question again

In [21]:
#llm_chat_request.documents = []
llm_chat_request.message = "tell me more about its databases"
llm_response = llm_client.chat(chat_detail)
llm_text = llm_response.data.chat_response.text
        
print (llm_text)

Oracle's database technology is widely recognized as the most flexible and cost-effective way to manage information and applications. It is a relational database management system (RDBMS) that is widely used for enterprise workloads. Here are some key points about Oracle databases:

1. **Relational Database Model**: Oracle databases are based on the relational database model, which organizes data into tables with rows and columns, ensuring data integrity and facilitating efficient data retrieval.

2. **Scalability**: They are known for their scalability, allowing businesses to handle small to very large data sets and support numerous concurrent users. Oracle databases can scale up to handle increased workloads and can also be scaled out in a distributed architecture.

3. **Security**: Oracle places a strong emphasis on security, offering features such as encryption, access control, and data redaction to protect sensitive information. It supports secure authentication methods and provid

*** print response & citations

In [22]:
print("**************************Chat Result**************************")
#llm_text = llm_response.data.chat_response.text
print(llm_response.data.chat_response.text)
print("************************** Citations**************************")
print(llm_response.data.chat_response.citations)

**************************Chat Result**************************
Oracle's database technology is widely recognized as the most flexible and cost-effective way to manage information and applications. It is a relational database management system (RDBMS) that is widely used for enterprise workloads. Here are some key points about Oracle databases:

1. **Relational Database Model**: Oracle databases are based on the relational database model, which organizes data into tables with rows and columns, ensuring data integrity and facilitating efficient data retrieval.

2. **Scalability**: They are known for their scalability, allowing businesses to handle small to very large data sets and support numerous concurrent users. Oracle databases can scale up to handle increased workloads and can also be scaled out in a distributed architecture.

3. **Security**: Oracle places a strong emphasis on security, offering features such as encryption, access control, and data redaction to protect sensitive i